In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import translitcodec
import codecs


In [ ]:
r = requests.get('https://www.wga.hu/cgi-bin/artist.cgi?Profession=any&School=any&Period=any&Time-line=any&from=0&max=50000&Sort=Name&letter=-&width=700&targetleft=0')

soup = BeautifulSoup(r.content, 'html.parser',from_encoding='latin-1')

In [2]:
tr = soup.find_all("tr")

In [4]:
res = tr[4].find_all('td', class_="ARTISTLIST")

In [6]:
out = []
for row in tr:
    cols = row.find_all('td',class_="ARTISTLIST")

    t = []
    print(cols)
    for col in cols:
        t.append(col.text.encode("latin-1").decode("utf-8"))
    out.append(t)

[]
[]
[<td class="ARTISTLIST"><a href="https://www.wga.hu/html/a/alvares/index.html"><b>ÃLVARES, Baltasar</b></a></td>, <td class="ARTISTLIST">(active 1570-1624)</td>, <td class="ARTISTLIST">Baroque</td>, <td class="ARTISTLIST">Portuguese architect </td>]
[<td class="ARTISTLIST"><a href="https://www.wga.hu/html/a/alvarez/index.html"><b>ÃLVAREZ Y CUBERO, JosÃ©</b></a></td>, <td class="ARTISTLIST">(1768-1827)</td>, <td class="ARTISTLIST">Neoclassicism</td>, <td class="ARTISTLIST">Spanish sculptor </td>]
[<td class="ARTISTLIST"><a href="https://www.wga.hu/html/s/slewinsk/index.html"><b>ÅLEWIÅSKI, WÅadisÅaw</b></a></td>, <td class="ARTISTLIST">(1856-1918)</td>, <td class="ARTISTLIST">Impressionism</td>, <td class="ARTISTLIST">Polish painter  (Paris)</td>]
[<td class="ARTISTLIST"><a href="https://www.wga.hu/html/a/aagaard/index.html"><b>AAGAARD, Carl Frederik</b></a></td>, <td class="ARTISTLIST">(1833-1895)</td>, <td class="ARTISTLIST">Realism</td>, <td class="ARTISTLIST">Danish paint

In [19]:
df = pd.DataFrame(out)
df.columns = ['Name','Period','Style','School']

In [32]:
df = df.dropna()

In [45]:
df['Name'] = df['Name'].map(lambda x: codecs.encode(x,'translit/short'))

C:\Users\romaf\AppData\Local\Temp\ipykernel_9332\3649483201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df['Name'].map(lambda x: codecs.encode(x,'translit/short'))


In [38]:
dfi = pd.read_table("img_catalog.txt",encoding='ansi')

In [41]:
dfi['AUTHOR'] = dfi['AUTHOR'].map(lambda x: codecs.encode(x,'translit/short'))

In [51]:
def flter_out(sent):
    fill = []
    for x in sent:
        if x == '(':
            break
        fill.append(x)
    return "".join(fill).strip()

df['Name']  = df['Name'].map(flter_out)

C:\Users\romaf\AppData\Local\Temp\ipykernel_9332\1994736541.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name']  = df['Name'].map(flter_out)


In [82]:
all_df = dfi.join(df.set_index('Name'),on='AUTHOR',how="left")

In [83]:
all_df = all_df.dropna()

In [84]:
all_df = all_df.loc[all_df['FORM'] == 'painting']

In [85]:
all_df.head()

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME,Period,Style,School
21,"AAGAARD, Carl Frederik","(b. 1833, Odense, d. 1895, København)",Deer beside a Lake,1888,"Oil on canvas, 53 x 82 cm",Private collection,https://www.wga.hu/html/a/aagaard/deerlake.html,painting,landscape,Danish,1851-1900,(1833-1895),Realism,Danish painter (Copenhagen)
22,"AAGAARD, Carl Frederik","(b. 1833, Odense, d. 1895, København)",The Rose Garden,1877,"Oil on canvas, 98 x 80 cm",Private collection,https://www.wga.hu/html/a/aagaard/rosegard.html,painting,landscape,Danish,1851-1900,(1833-1895),Realism,Danish painter (Copenhagen)
23,"ABADIA, Juan de la",(active 1470-1490 in Huesca),The Archangel Michael,c. 1490,"Wood, 127 x 78 cm","Museu Nacional d'Art de Catalunya, Barcelona",https://www.wga.hu/html/a/abadia/michael.html,painting,religious,Spanish,1451-1500,(active 1470-1490),Early Renaissance,Spanish painter (Huesca)
34,"ABBATE, Niccolo dell'","(b. 1509, Modena, d. 1571, Fontainebleau)",Chimney breast,c. 1552,Oil on canvas,"Château, Écouen",https://www.wga.hu/html/a/abbate/chimney1.html,painting,other,Italian,1501-1550,(1509-1571),Mannerism,Italian painter (Bologna)
36,"ABBATE, Niccolo dell'","(b. 1509, Modena, d. 1571, Fontainebleau)",Stag Hunt,1550-52,"Oil on canvas, 116 x 159 cm","Galleria Borghese, Rome",https://www.wga.hu/html/a/abbate/deerhunt.html,painting,landscape,Italian,1501-1550,(1509-1571),Mannerism,Italian painter (Bologna)


In [86]:
all_df = all_df[["AUTHOR",'TITLE','DATE',"URL","TYPE","SCHOOL","Style",'Period']]
all_df.columns = [x.lower() for x in["AUTHOR",'TITLE','DATE',"URL","TYPE","SCHOOL","Style",'Period']]

In [87]:
all_df.head()

,author,title,date,url,type,school,style,period
21,"AAGAARD, Carl Frederik",Deer beside a Lake,1888,https://www.wga.hu/html/a/aagaard/deerlake.html,landscape,Danish,Realism,(1833-1895)
22,"AAGAARD, Carl Frederik",The Rose Garden,1877,https://www.wga.hu/html/a/aagaard/rosegard.html,landscape,Danish,Realism,(1833-1895)
23,"ABADIA, Juan de la",The Archangel Michael,c. 1490,https://www.wga.hu/html/a/abadia/michael.html,religious,Spanish,Early Renaissance,(active 1470-1490)
34,"ABBATE, Niccolo dell'",Chimney breast,c. 1552,https://www.wga.hu/html/a/abbate/chimney1.html,other,Italian,Mannerism,(1509-1571)
36,"ABBATE, Niccolo dell'",Stag Hunt,1550-52,https://www.wga.hu/html/a/abbate/deerhunt.html,landscape,Italian,Mannerism,(1509-1571)


In [74]:
import regex as re

In [113]:
def parse_date(x):
    res = re.search(r'\d{4}',x)
    if not res:
        return(None)
    else:
        return res[0]

def parse_period(x):
    res = re.findall(r'\d{4}',x)
    if not res:
        return(None)
    else:
        if len(res) == 1:
            return res[0]
        return (int(res[0]) + int(res[1]) ) // 2

In [115]:
all_df['estimate_date'] = all_df['period'].map(parse_period) 

In [93]:
all_df['date'] = all_df['date'].map(parse_date)

In [121]:
all_df['date'] = all_df['date'].fillna(all_df['estimate_date'])

In [127]:
all_df = all_df[['author','title','date','url','type','style','school']]
all_df.head()

,author,title,date,url,type,style,school
21,"AAGAARD, Carl Frederik",Deer beside a Lake,1888,https://www.wga.hu/html/a/aagaard/deerlake.html,landscape,Realism,Danish
22,"AAGAARD, Carl Frederik",The Rose Garden,1877,https://www.wga.hu/html/a/aagaard/rosegard.html,landscape,Realism,Danish
23,"ABADIA, Juan de la",The Archangel Michael,1490,https://www.wga.hu/html/a/abadia/michael.html,religious,Early Renaissance,Spanish
34,"ABBATE, Niccolo dell'",Chimney breast,1552,https://www.wga.hu/html/a/abbate/chimney1.html,other,Mannerism,Italian
36,"ABBATE, Niccolo dell'",Stag Hunt,1550,https://www.wga.hu/html/a/abbate/deerhunt.html,landscape,Mannerism,Italian


In [134]:
all_df['url'] = all_df['url'].map(lambda x : x.replace("html",'art')[:-3] + 'jpg')

In [135]:
all_df.head()

,author,title,date,url,type,style,school
21,"AAGAARD, Carl Frederik",Deer beside a Lake,1888,https://www.wga.hu/art/a/aagaard/deerlake.jpg,landscape,Realism,Danish
22,"AAGAARD, Carl Frederik",The Rose Garden,1877,https://www.wga.hu/art/a/aagaard/rosegard.jpg,landscape,Realism,Danish
23,"ABADIA, Juan de la",The Archangel Michael,1490,https://www.wga.hu/art/a/abadia/michael.jpg,religious,Early Renaissance,Spanish
34,"ABBATE, Niccolo dell'",Chimney breast,1552,https://www.wga.hu/art/a/abbate/chimney1.jpg,other,Mannerism,Italian
36,"ABBATE, Niccolo dell'",Stag Hunt,1550,https://www.wga.hu/art/a/abbate/deerhunt.jpg,landscape,Mannerism,Italian


In [140]:
all_df = all_df.reset_index(drop=True)

In [141]:
all_df.to_csv("art_data.csv")